# Project Desciption

This project explores the application of Reinforcement Learning (RL) and Inverse Reinforcement Learning (IRL) to optimize bus transit schedules using real-world transit data. The primary objective is to develop a predictive model that can generate efficient bus schedules, reduce delays, and improve the reliability of public transportation services. By combining RL, which learns optimal actions based on rewards, with IRL, which derives rewards from observed behavior, this model aims to achieve a balance between planned schedules and realistic, adaptive operations based on historical patterns. The project utilizes data from the New York City MTA-GTFS bus line, modeling bus behavior and transit flow to simulate and predict optimal routes.

# Import Necessary Packages and Unzip the Data

* Description: In this segment, we install and import the necessary libraries for data analysis, reinforcement learning, and handling of neural networks. Essential libraries include stable-baselines3 for RL models, torch for deep learning, and pandas for data handling. We then extract the compressed data file containing historical bus transit records, which will serve as the basis for training and testing the models.
* Purpose: Set up the working environment with all dependencies to ensure smooth operation throughout the analysis and modeling stages.

In [18]:
!pip install stable-baselines3[extra]
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import zipfile
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import gym
from gym import spaces
from torch.distributions import Categorical
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.callbacks import EvalCallback

# Unzip the data
with zipfile.ZipFile('/content/mta_bus_data.zip', 'r') as zip_ref:
    zip_ref.extractall('.')

# Data Loading

* Description: Here, we load the bus transit dataset into a pandas DataFrame for processing. The dataset contains information such as vehicle IDs, timestamps, stop sequences, distances traveled, and stop IDs. Each entry represents a record of a bus’s journey at a specific time.
* Purpose: This segment is essential for loading the raw data, which provides the foundational information needed for transit modeling and allows us to inspect the structure and contents of the dataset.

In [19]:
df = pd.read_csv('/content/B63-2011-04-03_2011-05-03.csv')
df

,vehicle_id,timestamp,latitude,longitude,phase,trip_id,direction_id,trip_headsign,shape_dist_traveled,stop_id,stop_sequence,dist_from_stop
0,7573,2011-04-16 01:02:41,40.612648,-74.033723,IN_PROGRESS,20110403AD_004000_B63_0089_B63_101,0,B63 PIER 6 BKLYN BRIDGE PK via 5 AV,299.381043,305334,2,39.875024
1,7573,2011-04-16 01:03:16,40.613071,-74.033353,IN_PROGRESS,20110403AD_004000_B63_0089_B63_101,0,B63 PIER 6 BKLYN BRIDGE PK via 5 AV,357.314663,305335,3,158.012705
2,7573,2011-04-16 01:03:50,40.614715,-74.032055,IN_PROGRESS,20110403AD_004000_B63_0089_B63_101,0,B63 PIER 6 BKLYN BRIDGE PK via 5 AV,570.261417,308130,4,173.253540
3,7573,2011-04-16 01:04:09,40.617474,-74.030490,IN_PROGRESS,20110403AD_004000_B63_0089_B63_101,0,B63 PIER 6 BKLYN BRIDGE PK via 5 AV,898.529204,305338,5,130.538314
4,7573,2011-04-16 01:04:40,40.621249,-74.028916,IN_PROGRESS,20110403AD_004000_B63_0089_B63_101,0,B63 PIER 6 BKLYN BRIDGE PK via 5 AV,1322.867866,305341,7,189.051827
...,...,...,...,...,...,...,...,...,...,...,...,...
840935,7586,2011-04-23 01:22:21,40.648513,-74.006835,IN_PROGRESS,20110403JA_149500_B63_0002_B63_129,1,B63 BAY RIDGE SHORE RD via 5 AV,6822.575276,308337,34,123.612851
840936,7586,2011-04-23 01:24:05,40.647063,-74.008340,IN_PROGRESS,20110403JA_149500_B63_0002_B63_129,1,B63 BAY RIDGE SHORE RD via 5 AV,7029.367300,305442,35,151.892403
840937,7586,2011-04-23 01:25:17,40.643003,-74.012593,IN_PROGRESS,20110403JA_149500_B63_0002_B63_129,1,B63 BAY RIDGE SHORE RD via 5 AV,7608.351514,305446,38,206.599373
840938,7586,2011-04-23 01:25:26,40.642570,-74.012877,IN_PROGRESS,20110403JA_149500_B63_0002_B63_129,1,B63 BAY RIDGE SHORE RD via 5 AV,7661.100290,305446,38,153.850597


# Data Preprocessing

* Description: This segment prepares the transit data for use in machine learning models by cleaning and engineering relevant features. Key steps include:
1.   Timestamp Standardization: Converts timestamps into a consistent format for time-based calculations.
2.   Sorting and Handling Missing Values: Organizes data sequentially by vehicle ID and timestamp, and fills missing values in columns like stop_id to maintain data continuity.
3.   Feature Engineering:
Calculates distance_traveled and speed between stops to capture movement characteristics.
Encodes bus phase statuses (e.g., "IN_PROGRESS") into numerical values, making them compatible with machine learning models.
Generates temporal features (e.g., hour of day, day of the week) to capture
potential patterns in transit behavior.

* Purpose: Data preprocessing is crucial for converting raw data into a structured format that machine learning models can interpret. This step enriches the dataset with additional features, improving model accuracy and enabling better predictions.

In [20]:
# Convert Timestamps: Parse the timestamp column to a standard datetime format, enabling sorting and time-based feature engineering.
# Sort by Vehicle ID and Time: Sort data by vehicle_id and timestamp to maintain sequential integrity for each bus, essential for trajectory tracking.
# Handle Missing Values: Fill missing values in columns like stop_id, stop_sequence, and dist_from_stop based on neighboring entries or interpolate where needed.
# Engineer Distance and Speed Features:
#     Calculate distance_traveled using shape_dist_traveled differences between consecutive rows per vehicle_id.
#     Derive speed based on time differences and distance, useful for analyzing delays and detecting congested routes.
# Categorize Phases: Convert the phase column to binary or numerical indicators (e.g., IN_PROGRESS = 1, LAYOVER_DURING = 0) to ease RL state encoding.
# Generate Sequential Trip Data: Group by trip_id and calculate statistics, such as average travel time per trip, providing a target for RL

def preprocess_data(df):
    """Preprocess the transit data for RL-IRL environments"""
    # Create a copy to avoid modifications to original data
    df = df.copy()

    # Convert timestamp to datetime
    df['timestamp'] = pd.to_datetime(df['timestamp'])

    # Sort by vehicle and time
    df = df.sort_values(['vehicle_id', 'timestamp'])

    # Forward fill missing values
    df['stop_id'] = df['stop_id'].ffill()
    df['stop_sequence'] = df['stop_sequence'].ffill()
    df['dist_from_stop'] = df['dist_from_stop'].ffill()

    # Calculate time differences and speeds
    df['time_diff'] = df.groupby('vehicle_id')['timestamp'].diff().dt.total_seconds()
    df['distance_traveled'] = df.groupby('vehicle_id')['shape_dist_traveled'].diff()

    # Calculate speed (meters per second)
    df['speed'] = (df['distance_traveled'] / df['time_diff']).replace([np.inf, -np.inf], np.nan)

    # Fill NaN values with appropriate defaults
    df['speed'] = df['speed'].fillna(0)
    df['time_diff'] = df['time_diff'].fillna(0)
    df['distance_traveled'] = df['distance_traveled'].fillna(0)

    # Convert phase to numerical
    phase_map = {'IN_PROGRESS': 1, 'LAYOVER_DURING': 0}
    df['phase'] = df['phase'].map(phase_map).fillna(-1)

    # Add temporal features
    df['hour'] = df['timestamp'].dt.hour
    df['day'] = df['timestamp'].dt.dayofweek
    df['week'] = df['timestamp'].dt.isocalendar().week

    return df

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [21]:
# Step 2: Feature Engineering and State Representation
# Now that the data is preprocessed, we can proceed with feature engineering:
# Key Steps for Step 2:
#     Define State Features: Use features like latitude, longitude, shape_dist_traveled, phase, and stop_id as core components of each bus’s state.
#     Create Time-Based Features: Break down timestamp into hour, day, and week features to capture temporal patterns, such as peak hours or weekday/weekend trends.
#     Normalize Numerical Features: Scale features like distance_traveled and speed to a range suitable for RL models.
#     Encode Categorical Data: Convert phase, direction_id, and trip_headsign into one-hot or embedding representations for easier integration into RL models.


def feature_engineer(df):
    """Engineer features for the RL environment"""
    processed_df = df.copy()

    # Numerical features to normalize
    numerical_cols = [
        'latitude', 'longitude', 'shape_dist_traveled',
        'dist_from_stop', 'distance_traveled', 'speed',
        'time_diff'
    ]

    # Normalize numerical features
    scaler = StandardScaler()
    processed_df[numerical_cols] = scaler.fit_transform(processed_df[numerical_cols])

    # One-hot encode categorical variables
    categorical_cols = ['direction_id', 'trip_headsign']
    processed_df = pd.get_dummies(processed_df, columns=categorical_cols, prefix=categorical_cols)

    # Create hour sine and cosine features for cyclical time representation
    processed_df['hour_sin'] = np.sin(2 * np.pi * processed_df['hour']/24)
    processed_df['hour_cos'] = np.cos(2 * np.pi * processed_df['hour']/24)

    # Create day sine and cosine features
    processed_df['day_sin'] = np.sin(2 * np.pi * processed_df['day']/7)
    processed_df['day_cos'] = np.cos(2 * np.pi * processed_df['day']/7)

    return processed_df

# Environment Setup for RL and IRL Modeling

* Description: In this part, we create a custom environment tailored for RL and IRL, which simulates the bus transit system. This environment allows RL agents to interact with the transit data, selecting actions that represent bus route decisions and observing the results. We define state representations (such as bus locations and times) and actions (e.g., adjust speed or direction).
* Purpose: Establishing an RL/IRL environment is fundamental for training agents on the transit data. It serves as a virtual platform where models learn to make optimal scheduling decisions based on reward functions, ultimately leading to efficient bus routing strategies.

## Environment Definition

In [22]:
class MultiBusRouteEnv(gym.Env):
    """Multi-bus transit system environment"""

    def __init__(self, data, time_limit=1000):
        super(MultiBusRouteEnv, self).__init__()

        # Preprocess and engineer features
        self.data = preprocess_data(data)
        self.processed_data = feature_engineer(self.data)

        # Group data by vehicle_id
        self.buses_data = dict(tuple(self.processed_data.groupby('vehicle_id')))
        self.bus_ids = list(self.buses_data.keys())
        self.num_buses = len(self.bus_ids)

        # Define feature columns for state space
        self.state_columns = [
            'latitude', 'longitude', 'shape_dist_traveled',
            'dist_from_stop', 'distance_traveled', 'speed',
            'hour_sin', 'hour_cos', 'day_sin', 'day_cos',
            'phase'
        ] + [col for col in self.processed_data.columns if 'direction_id_' in col or 'trip_headsign_' in col]

        # Flattened action space for all buses: 2 actions (speed adjustment, stop duration adjustment) per bus
        self.action_space = spaces.Box(
            low=-1,
            high=1,
            shape=(self.num_buses * 2,),  # 2 actions per bus
            dtype=np.float32
        )

        # Dynamically set the observation space based on state columns and nearby bus features
        num_features = len(self.state_columns) + (self.num_buses - 1) * 4  # Additional features for nearby buses
        self.observation_space = spaces.Box(
            low=-np.inf,
            high=np.inf,
            shape=(self.num_buses, num_features),
            dtype=np.float32
        )

        self.time_limit = time_limit
        self.reset()

    def get_bus_state(self, bus_id):
        """Get current state for a specific bus including nearby bus information"""
        current_state = self.current_states[bus_id]

        # Basic state features
        state_values = current_state[self.state_columns].values.astype(np.float32)

        # Get nearby bus information
        current_pos = np.array([current_state['latitude'], current_state['longitude']])
        nearby_features = []

        for other_id, other_state in self.current_states.items():
            if other_id != bus_id and len(nearby_features) < (self.num_buses - 1) * 4:  # 4 features per nearby bus
                other_pos = np.array([other_state['latitude'], other_state['longitude']])
                relative_pos = other_pos - current_pos
                relative_speed = other_state['speed'] - current_state['speed']

                nearby_features.extend([
                    relative_pos[0],
                    relative_pos[1],
                    relative_speed,
                    other_state['phase']
                ])

        # Ensure we pad nearby_features to have all buses worth of data
        while len(nearby_features) < (self.num_buses - 1) * 4:
            nearby_features.extend([0, 0, 0, 0])

        # Combine main state features and nearby bus features
        combined_state = np.concatenate([state_values, nearby_features])

        return combined_state

    def calculate_reward(self, bus_id, state, action, next_state):
      """Calculate reward for a single bus"""

      # Initialize rewards as a dictionary
      rewards = {
          'schedule_adherence': 0,
          'speed_efficiency': 0,
          'passenger_comfort': 0,
          'bus_spacing': 0
      }

      # Schedule adherence reward
      stop_deviation = abs(next_state['dist_from_stop'] - state['dist_from_stop'])
      rewards['schedule_adherence'] = -np.clip(stop_deviation / 100, 0, 1)

      # Speed efficiency reward
      optimal_speed = 10  # Hyperparameter: meters per second (about 22 mph)
      speed_deviation = abs(next_state['speed'] - optimal_speed)
      rewards['speed_efficiency'] = -np.clip(speed_deviation / optimal_speed, 0, 1)

      # Passenger comfort reward (penalize jerky movements)
      speed_change = abs(next_state['speed'] - state['speed'])
      rewards['passenger_comfort'] = -np.clip(speed_change / 2, 0, 1)

      # Bus spacing reward
      nearest_distance = float('inf')
      for other_id, other_state in self.current_states.items():
          if other_id != bus_id:
              dist = np.sqrt(
                  (next_state['latitude'] - other_state['latitude'])**2 +
                  (next_state['longitude'] - other_state['longitude'])**2
              )
              nearest_distance = min(nearest_distance, dist)

      optimal_spacing = 0.01  # Approximately 1km in normalized coordinates
      spacing_deviation = abs(nearest_distance - optimal_spacing)
      rewards['bus_spacing'] = -np.clip(spacing_deviation / optimal_spacing, 0, 1)

      # Weighted sum of rewards
      weights = {
          'schedule_adherence': 0.3,
          'speed_efficiency': 0.3,
          'passenger_comfort': 0.2,
          'bus_spacing': 0.2
      }

      # Calculate total reward by applying weights to the individual reward components
      total_reward = sum(rewards[key] * weights[key] for key in rewards)

      # Ensure total_reward is a scalar value
      return total_reward

    def step(self, actions):
      """Environment step for all buses"""
      self.steps_taken += 1
      rewards = {}
      dones = {}
      infos = {}

      # Split flattened action array back into individual bus actions
      actions = np.split(actions, self.num_buses)

      # Store current states for reward calculation
      previous_states = self.current_states.copy()

      # Update states for all buses
      for i, bus_id in enumerate(self.bus_ids):
          bus_id_str = str(bus_id)
          action = actions[i]

          current_idx = self.bus_indices[bus_id]
          bus_data = self.buses_data[bus_id]

          # Update index and get next state
          self.bus_indices[bus_id] = (current_idx + 1) % len(bus_data)
          next_state = bus_data.iloc[self.bus_indices[bus_id]].copy()

          # Apply speed adjustment
          speed_adjustment = action[0] * 2  # Scale to reasonable speed change
          next_state['speed'] += speed_adjustment
          next_state['speed'] = np.clip(next_state['speed'], 0, 20)  # Max 20 m/s

          # Update current states
          self.current_states[bus_id] = next_state

          # Calculate rewards
          rewards[bus_id_str] = self.calculate_reward(
              bus_id,
              previous_states[bus_id],
              action,
              next_state
          )

          dones[bus_id_str] = False
          infos[bus_id_str] = {}

      # Global done condition
      dones['__all__'] = self.steps_taken >= self.time_limit

      # Collect observations for all buses
      observations = []
      for bus_id in self.bus_ids:
          bus_state = self.get_bus_state(bus_id)
          if bus_state.shape[0] != self.observation_space.shape[1]:
              raise ValueError(f"Bus state shape {bus_state.shape} does not match expected feature size {self.observation_space.shape[1]}")
          observations.append(bus_state)
      observations = np.stack(observations)

      # Ensure observation shape consistency
      assert observations.shape == self.observation_space.shape, \
          f"Observation shape {observations.shape} does not match expected shape {self.observation_space.shape}"

      return observations, np.array(list(rewards.values())).sum(), dones, infos

    def reset(self):
        """Reset environment"""
        self.steps_taken = 0
        self.bus_indices = {bus_id: 0 for bus_id in self.bus_ids}
        self.current_states = {
            bus_id: self.buses_data[bus_id].iloc[0]
            for bus_id in self.bus_ids
        }

        observations = np.array([
            self.get_bus_state(bus_id)
            for bus_id in self.bus_ids
        ])
        return observations


## IRL Model Definition

In [23]:
class TransitFeatureExtractor(nn.Module):
    """Neural network for extracting features from transit state-action pairs"""
    def __init__(self):
        super(TransitFeatureExtractor, self).__init__()

        # Input dimensions based on your columns
        self.continuous_features = 8  # lat, long, shape_dist, dist_from_stop,
                                    # distance_traveled, time_diff, speed, total_trip_time
        self.phase_features = 3      # phase_0.0, phase_1.0, phase_nan
        self.direction_features = 3  # direction_id_0.0, direction_id_1.0, direction_id_nan
        self.headsign_features = 4   # All trip_headsign variants

        # Temporal features
        self.temporal_features = 3   # hour, day, week

        # Calculate total input dimension
        self.input_dim = (self.continuous_features + self.phase_features +
                         self.direction_features + self.headsign_features +
                         self.temporal_features)

        # Action dimension (from your RL environment: speed adjustment and stop duration)
        self.action_dim = 2

        # Feature extraction network
        self.network = nn.Sequential(
            nn.Linear(self.input_dim + self.action_dim, 256),
            nn.ReLU(),
            nn.BatchNorm1d(256),
            nn.Dropout(0.3),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.BatchNorm1d(128),
            nn.Dropout(0.2),
            nn.Linear(128, 64)
        )

    def forward(self, states, actions):
        combined = torch.cat([states, actions], dim=-1)
        return self.network(combined)

class TransitStateProcessor:
    """Process raw transit state data into tensor format"""
    def __init__(self):
        self.continuous_scaler = StandardScaler()
        self.temporal_scaler = StandardScaler()

    def fit(self, data):
        """Fit scalers to training data"""
        continuous_cols = [
            'latitude', 'longitude', 'shape_dist_traveled',
            'dist_from_stop', 'distance_traveled', 'time_diff',
            'speed', 'total_trip_time'
        ]
        temporal_cols = ['hour', 'day', 'week']

        self.continuous_scaler.fit(data[continuous_cols])
        self.temporal_scaler.fit(data[temporal_cols])

    def process_state(self, state):
        """Convert a single state to tensor format"""
        # Extract and scale continuous features
        continuous_features = np.array([
            state['latitude'], state['longitude'],
            state['shape_dist_traveled'], state['dist_from_stop'],
            state['distance_traveled'], state['time_diff'],
            state['speed'], state['total_trip_time']
        ]).reshape(1, -1)

        continuous_scaled = self.continuous_scaler.transform(continuous_features)

        # Extract and scale temporal features
        temporal_features = np.array([
            state['hour'], state['day'], state['week']
        ]).reshape(1, -1)

        temporal_scaled = self.temporal_scaler.transform(temporal_features)

        # Extract categorical features
        phase_features = np.array([
            state['phase_0.0'], state['phase_1.0'], state['phase_nan']
        ])

        direction_features = np.array([
            state['direction_id_0.0'], state['direction_id_1.0'],
            state['direction_id_nan']
        ])

        headsign_features = np.array([
            state['trip_headsign_B63 39 ST - 5 AV'],
            state['trip_headsign_B63 BAY RIDGE SHORE RD via 5 AV'],
            state['trip_headsign_B63 PIER 6 BKLYN BRIDGE PK via 5 AV'],
            state['trip_headsign_nan']
        ])

        # Combine all features
        combined_features = np.concatenate([
            continuous_scaled.flatten(),
            temporal_scaled.flatten(),
            phase_features,
            direction_features,
            headsign_features
        ])

        return torch.FloatTensor(combined_features)

class MultiBusMaxEntIRL:
    """MaxEnt IRL implementation specifically for transit system"""

    def __init__(self, env, expert_trajectories, learning_rate=0.001):
        self.env = env
        self.expert_trajectories = expert_trajectories
        self.state_processor = TransitStateProcessor()

        # Initialize models for each bus
        self.feature_extractors = {}
        self.reward_networks = {}
        self.optimizers = {}

        for bus_id in env.bus_ids:
            self.feature_extractors[bus_id] = TransitFeatureExtractor()

            # Reward network with transit-specific architecture
            self.reward_networks[bus_id] = nn.Sequential(
                nn.Linear(64, 32),
                nn.ReLU(),
                nn.BatchNorm1d(32),
                nn.Linear(32, 16),
                nn.ReLU(),
                nn.BatchNorm1d(16),
                nn.Linear(16, 1)
            )

            # Combine parameters and create optimizer
            parameters = list(self.feature_extractors[bus_id].parameters()) + \
                        list(self.reward_networks[bus_id].parameters())

            self.optimizers[bus_id] = optim.Adam(
                parameters,
                lr=learning_rate,
                weight_decay=1e-5
            )

        # Fit state processor to expert data
        self.fit_state_processor()

    def fit_state_processor(self):
        """Fit the state processor to expert trajectories"""
        all_states = []
        for bus_trajectories in self.expert_trajectories.values():
            for trajectory in bus_trajectories:
                all_states.extend(trajectory['states'])

        self.state_processor.fit(pd.DataFrame(all_states))

    def process_trajectory(self, trajectory):
        """Process a trajectory into tensor format"""
        processed_states = torch.stack([
            self.state_processor.process_state(state)
            for state in trajectory['states']
        ])

        processed_actions = torch.FloatTensor(trajectory['actions'])
        return processed_states, processed_actions

    def compute_reward(self, states, actions, bus_id):
        """Compute reward with transit-specific features"""
        features = self.feature_extractors[bus_id](states, actions)
        raw_rewards = self.reward_networks[bus_id](features)

        # Add transit-specific reward shaping
        schedule_adherence = torch.abs(states[:, 3])  # dist_from_stop
        speed_efficiency = torch.abs(states[:, 6] - 10.0)  # deviation from optimal speed

        shaped_rewards = raw_rewards - 0.1 * schedule_adherence - 0.05 * speed_efficiency
        return shaped_rewards

    def train(self, policy, bus_id, num_iterations=100):
        """MaxEnt IRL training with transit-specific considerations"""
        # Process expert trajectories
        expert_trajectories = self.expert_trajectories[bus_id]
        all_expert_states = []
        all_expert_actions = []

        for trajectory in expert_trajectories:
            states, actions = self.process_trajectory(trajectory)
            all_expert_states.append(states)
            all_expert_actions.append(actions)

        expert_states = torch.cat(all_expert_states, dim=0)
        expert_actions = torch.cat(all_expert_actions, dim=0)

        for iteration in range(num_iterations):
            # Forward pass with expert data
            expert_features = self.feature_extractors[bus_id](
                expert_states,
                expert_actions
            )
            expert_rewards = self.reward_networks[bus_id](expert_features)

            # Generate policy trajectories
            policy_states = []
            policy_actions = []

            state = self.env.reset()[str(bus_id)]
            done = False

            while not done:
                processed_state = self.state_processor.process_state(state)
                action = policy.predict(state)[0]

                policy_states.append(processed_state)
                policy_actions.append(torch.FloatTensor(action))

                next_state, _, done, _ = self.env.step({str(bus_id): action})
                state = next_state[str(bus_id)]

            policy_states = torch.stack(policy_states)
            policy_actions = torch.stack(policy_actions)

            # Compute policy features and rewards
            policy_features = self.feature_extractors[bus_id](
                policy_states,
                policy_actions
            )
            policy_rewards = self.reward_networks[bus_id](policy_features)

            # Compute MaxEnt IRL loss with regularization
            loss = -(torch.mean(expert_rewards) - torch.mean(policy_rewards))

            # Add L2 regularization
            l2_lambda = 0.01
            l2_norm = sum(p.pow(2.0).sum() for p in self.feature_extractors[bus_id].parameters())
            loss = loss + l2_lambda * l2_norm

            # Backward pass
            self.optimizers[bus_id].zero_grad()
            loss.backward()
            self.optimizers[bus_id].step()

            if (iteration + 1) % 10 == 0:
                print(f"Bus {bus_id} - Iteration {iteration + 1}/{num_iterations}, "
                      f"Loss: {loss.item():.4f}")

# Defining RL and IRL Models

* Description: Here, we define both RL and IRL models, each with distinct objectives:

1.   RL Model: Learns from a reward function to make real-time adjustments to bus schedules based on performance metrics such as minimizing delay. PPO(Proximal Policy Optimization) RL algorithm is used to accomplish the task.
2.   IRL Model: Observes historical behavior from the dataset to infer a reward structure, which guides buses to follow patterns that align with typical transit schedules. MaxEntIRL (Maximum Entropy Inverse RL) is used to accomplish the task.

* Purpose: Integrating RL and IRL models enables the system to balance schedule adherence with adaptability. The RL model adjusts based on current conditions, while the IRL model keeps operations close to realistic, observed behavior.

In [24]:
def train_and_evaluate_rl_models(data):
    """Train and evaluate the multi-bus optimization model"""
    # Create environment
    env = MultiBusRouteEnv(data)
    env = DummyVecEnv([lambda: env])

    # Create PPO model for each bus
    models = {}
    for bus_id in df['vehicle_id'].unique():
        models[str(bus_id)] = PPO(
            "MlpPolicy",
            env,
            verbose=1,
            learning_rate=3e-4,
            n_steps=2048,
            batch_size=64,
            n_epochs=10,
            gamma=0.99,
            gae_lambda=0.95,
            clip_range=0.2,
            ent_coef=0.01,
            policy_kwargs=dict(
                net_arch=[dict(pi=[128, 64], vf=[128, 64])],
                activation_fn=torch.nn.ReLU,
                ortho_init=True
            )
        )

    # Train each model
    for bus_id, model in models.items():
        eval_callback = EvalCallback(
            env,
            best_model_save_path=f"./logs/bus_{bus_id}/",
            log_path=f"./logs/bus_{bus_id}/",
            eval_freq=1000,
            deterministic=True,
            render=False
        )

        model.learn(total_timesteps=100000, callback=eval_callback)
        model.save(f"bus_route_model_{bus_id}_final")

    return models, env

In [ ]:
# Train RL model
rl_models, env = train_and_evaluate_rl_models(df)

# Create expert trajectories from the processed data
expert_trajectories = {}
for vehicle_id, group in df.groupby('vehicle_id'):
    # Create the list of states
    states = group[['latitude', 'longitude', 'shape_dist_traveled', 'speed', 'stop_id',
                    'dist_from_stop', 'distance_traveled', 'time_diff', 'total_trip_time',
                    'hour', 'day', 'week', 'phase_0.0', 'phase_1.0', 'phase_nan',
                    'direction_id_0.0', 'direction_id_1.0', 'direction_id_nan',
                    'trip_headsign_B63 39 ST - 5 AV', 'trip_headsign_B63 BAY RIDGE SHORE RD via 5 AV',
                    'trip_headsign_B63 PIER 6 BKLYN BRIDGE PK via 5 AV', 'trip_headsign_nan']].to_dict(orient='records')

    # Extract actions from the df
    actions = group[['speed', 'total_trip_time']].values.tolist()

    # Store in the expert_trajectories dictionary
    expert_trajectories[vehicle_id] = [{
        'states': states,
        'actions': actions
    }]


# Train IRL model
irl_model = MultiBusMaxEntIRL(env, expert_trajectories)
for bus_id in df['vehicle_id'].unique():
  irl_model.train(rl_models[bus_id], bus_id)

/usr/local/lib/python3.10/dist-packages/stable_baselines3/common/vec_env/patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/stable_baselines3/common/policies.py:486: UserWarning: As shared layers in the mlp_extractor are removed since SB3 v1.8.0, you should now pass directly a dictionary and not a list (net_arch=dict(pi=..., vf=...) instead of net_arch=[dict(pi=..., vf=...)])
  warnings.warn(


Using cpu device
Using cpu device
Using cpu device
Using cpu device
Using cpu device
Using cpu device
Using cpu device
Using cpu device
Using cpu device
Using cpu device
Using cpu device
Using cpu device
Using cpu device
Using cpu device
Using cpu device
Using cpu device
Using cpu device
Using cpu device
Using cpu device
Using cpu device
Using cpu device
Using cpu device
Using cpu device
Using cpu device
Using cpu device
Using cpu device
Using cpu device
Using cpu device
Using cpu device
Using cpu device


/usr/local/lib/python3.10/dist-packages/stable_baselines3/common/evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


Eval num_timesteps=1000, episode_reward=-14.09 +/- 0.00
Episode length: 1.00 +/- 0.00
---------------------------------
| eval/              |          |
|    mean_ep_length  | 1        |
|    mean_reward     | -14.1    |
| time/              |          |
|    total_timesteps | 1000     |
---------------------------------
New best mean reward!


# Combining RL and IRL Models

* Description: This section combines the decision-making policies of RL and IRL models to form a unified strategy. The combine_rl_irl_transit function merges actions from both models, with weights that dynamically adjust based on the bus’s deviation from its planned schedule. When far from the planned schedule, the RL policy is prioritized; when closer, IRL behavior dominates.
* Purpose: This combined approach leverages the strengths of both RL and IRL, balancing real-time adjustments with adherence to historical patterns. This results in more flexible yet predictable bus operations, improving schedule reliability and minimizing delays.

In [ ]:
def combine_rl_irl_transit(rl_models, irl_model, env, alpha=0.5):
    """Combine RL and IRL models with transit-specific considerations"""

    class TransitCombinedPolicy:
        def __init__(self, rl_model, reward_function, alpha):
            self.rl_model = rl_model
            self.reward_function = reward_function
            self.alpha = alpha
            self.state_processor = irl_model.state_processor

        def predict(self, state, deterministic=True):
            # Get RL model's action
            rl_action, _ = self.rl_model.predict(state, deterministic=deterministic)

            # Process state for IRL evaluation
            processed_state = self.state_processor.process_state(state)

            # Generate additional candidate actions around RL action
            candidate_actions = [
                rl_action,
                rl_action * 0.8,  # Slower option
                rl_action * 1.2   # Faster option
            ]

            # Evaluate all actions
            combined_scores = []
            for action in candidate_actions:
                # Get RL score
                rl_score = self.rl_model.policy.evaluate_actions(
                    torch.FloatTensor(state).unsqueeze(0),
                    torch.FloatTensor(action).unsqueeze(0)
                )[0].item()

                # Get IRL score
                irl_score = self.reward_function(
                    processed_state.unsqueeze(0),
                    torch.FloatTensor(action).unsqueeze(0)
                ).item()

                # Combine scores with transit-specific weighting
                schedule_deviation = abs(state['dist_from_stop'])
                if schedule_deviation > 100:  # If far from schedule
                    alpha = max(0.7, self.alpha)  # Increase RL weight
                else:
                    alpha = self.alpha

                combined_score = alpha * rl_score + (1 - alpha) * irl_score
                combined_scores.append(combined_score)

            # Select best action
            best_idx = np.argmax(combined_scores)
            return candidate_actions[best_idx]

    # Create combined policies
    combined_policies = {}
    for bus_id in env.bus_ids:
        reward_function = lambda s, a, bid=bus_id: irl_model.compute_reward(s, a, bid)
        combined_policies[bus_id] = TransitCombinedPolicy(
            rl_models[bus_id],
            reward_function,
            alpha
        )

    return combined_policies

# Predict the Optimal Policy

* Description: This section builds a MultiBusPolicyPredictor, a model that can predict optimal actions for multiple buses using the combined RL and IRL policies. It evaluates the models’ effectiveness in improving bus schedules by running simulations and comparing performance metrics like delay reduction and adherence to the schedule.
* Purpose: By testing and evaluating the combined policy on multiple bus lines, we can assess the effectiveness of our model in real-world applications. This predictor offers insights into how well the combined approach can generalize across different transit lines, highlighting areas for future model improvement.

In [ ]:
class MultiBusPolicyPredictor:
    """Predict actions for a multi-bus transit system using combined RL and IRL policies."""

    def __init__(self, env, combined_policies):
        """
        Initializes the policy predictor.

        Args:
            env: The transit environment containing state and bus information.
            combined_policies: A dictionary of TransitCombinedPolicy objects for each bus.
        """
        self.env = env
        self.combined_policies = combined_policies  # Combined policies for each bus

    def predict_action(self, bus_id, state):
        """
        Predict the best action for a specific bus using the combined RL-IRL policy.

        Args:
            bus_id: The ID of the bus for which to predict the action.
            state: The current state of the bus.

        Returns:
            The selected action for the bus.
        """
        policy = self.combined_policies[bus_id]
        action = policy.predict(state)
        return action

    def predict_all_actions(self, state):
        """
        Predict actions for all buses in the current environment state.

        Args:
            state: The current state of the environment containing all bus states.

        Returns:
            An array of actions, one for each bus in the environment.
        """
        actions = []
        for bus_id in self.env.bus_ids:
            bus_state = state[bus_id]
            action = self.predict_action(bus_id, bus_state)
            actions.append(action)
        return np.array(actions)

    def evaluate_policy(self, num_episodes=10):
        """
        Evaluate the combined RL-IRL policy over multiple episodes.

        Args:
            num_episodes: Number of episodes to run for evaluation.

        Returns:
            The mean and standard deviation of rewards per episode.
        """
        rewards_per_episode = []
        for _ in range(num_episodes):
            total_reward = 0
            state = self.env.reset()
            done = False
            while not done:
                actions = self.predict_all_actions(state)
                state, reward, done, _ = self.env.step(actions)
                total_reward += reward
            rewards_per_episode.append(total_reward)
        return np.mean(rewards_per_episode), np.std(rewards_per_episode)

In [ ]:
combined_policies = combine_rl_irl_transit(rl_models, irl_model, env, alpha=0.5)
policy_predictor = MultiBusPolicyPredictor(env, combined_policies)

# Predict actions for a given state
state = env.reset()
actions = policy_predictor.predict_all_actions(state)
print("Predicted Actions:", actions)

# Evaluate the combined policy
mean_reward, std_reward = policy_predictor.evaluate_policy(num_episodes=10)
print("Mean Reward:", mean_reward)
print("Reward Std Dev:", std_reward)

# Performance Metrics and Analytics

* Description: This final segment assesses the model’s performance using various metrics, such as average delay, schedule deviation, and efficiency scores. It may include plotting results to visualize the effect of the combined policy on bus punctuality and route efficiency.
* Purpose: Performance metrics are essential for evaluating the success of the model. By understanding how well the combined policy reduces delays and improves schedule adherence, we can quantify the impact of the RL and IRL approach on transit optimization.

In [ ]:
def extract_true_actions_and_timestamps(df):
    """
    Extracts the true actions and timestamps from the bus data DataFrame.

    Args:
    - df (pd.DataFrame): DataFrame containing bus route data.

    Returns:
    - true_actions (list): List of true action values based on `dist_from_stop`.
    - timestamps (list): List of timestamps associated with each action.
    """
    # Sort by timestamp to ensure chronological order
    df = df.sort_values(by='timestamp')

    # Extract true actions and timestamps
    true_actions = df['dist_from_stop'].tolist()
    timestamps = pd.to_datetime(df['timestamp']).tolist()

    return true_actions, timestamps


def evaluate_policy_performance(predicted_actions, true_actions, timestamps):
    """
    Evaluate the performance of the combined RL-IRL policy.

    Args:
    - predicted_actions: A list of actions predicted by the combined policy for each time step.
    - true_actions: A list of true actions (or ideal actions) for each time step.
    - timestamps: A list of timestamps associated with each action.

    Returns:
    - performance_metrics: A dictionary containing calculated performance metrics.
    - deviations: A list of schedule deviations over time.
    """
    # Calculate absolute schedule deviations (difference between predicted and true actions)
    deviations = [abs(pred - true) for pred, true in zip(predicted_actions, true_actions)]

    # Calculate average delay (average absolute difference)
    avg_delay = np.mean(deviations)

    # Calculate max deviation for severe cases
    max_deviation = np.max(deviations)

    # Calculate efficiency score as inverse of average delay
    efficiency_score = 1 / (1 + avg_delay)  # Higher score is better

    # Compile metrics
    performance_metrics = {
        'Average Delay': avg_delay,
        'Max Deviation': max_deviation,
        'Efficiency Score': efficiency_score
    }

    return performance_metrics, deviations

def plot_performance_metrics(deviations, timestamps):
    """
    Plot the deviations and analyze the effectiveness of the combined policy.

    Args:
    - deviations: List of schedule deviations for each timestamp.
    - timestamps: List of timestamps associated with each deviation.
    """
    plt.figure(figsize=(12, 6))

    # Plot deviations over time
    plt.plot(timestamps, deviations, label='Schedule Deviation', color='blue', linestyle='--')
    plt.xlabel('Timestamp')
    plt.ylabel('Deviation (from ideal schedule)')
    plt.title('Schedule Deviation Over Time')
    plt.legend()
    plt.show()

# Extract true actions and timestamps from dataframe
true_actions, timestamps = extract_true_actions_and_timestamps(df)

# Compute performance metrics
performance_metrics, deviations = evaluate_policy_performance(actions, true_actions, timestamps)

# Display performance metrics
print("Performance Metrics:")
for metric, value in performance_metrics.items():
    print(f"{metric}: {value:.2f}")

# Plot schedule deviations over time
plot_performance_metrics(deviations, timestamps)